In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
DATA_DIR = '../input/'

In [4]:
import os

print(os.listdir(DATA_DIR))

In [5]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as T

image_size = 64
batch_size = 128
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

In [6]:
train_ds = ImageFolder(DATA_DIR, transform=T.Compose([
    T.Resize(image_size),
    T.CenterCrop(image_size),
    T.ToTensor(),
    T.Normalize(*stats)]))

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)

In [7]:
train_ds[0]

In [8]:
def denorm(x):
    return x*stats[1][0] + stats[0][0]

In [9]:
import torch
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
def show_images(images, nmax=64):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(make_grid(denorm(images.detach()[:nmax]), nrow=8).permute(1, 2, 0))

def show_batch(dl, nmax=64):
    for images, _ in dl:
        show_images(images, nmax)
        break

In [11]:
show_batch(train_dl)

In [12]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [13]:
device = get_default_device()
device

In [14]:
train_dl = DeviceDataLoader(train_dl, device)

In [15]:
import torch.nn as nn

In [16]:
discriminator = nn.Sequential(
        nn.Conv2d(3,64, kernel_size=4, stride=2, padding=1,bias=False),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(0.2),
        
        nn.Conv2d(64,128, kernel_size=4, stride=2, padding=1, bias=False),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(0.2),
    
        nn.Conv2d(128,256, kernel_size=4, stride=2, padding=1, bias=False),
        nn.BatchNorm2d(256),
        nn.LeakyReLU(0.2),
    
        nn.Conv2d(256,512, kernel_size=4, stride=2, padding=1, bias=False),
        nn.BatchNorm2d(512),
        nn.LeakyReLU(0.2),
    
        nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),
        nn.Flatten(),
        nn.Sigmoid()
)

In [17]:
discriminator = to_device(discriminator,device)
discriminator

In [18]:
latent_size=128

In [19]:
generator = nn.Sequential(
    
        nn.ConvTranspose2d(latent_size, 512, kernel_size=4, stride=1, padding=0, bias=False),
        nn.BatchNorm2d(512),
        nn.ReLU(True),
    
        nn.ConvTranspose2d(512,256,kernel_size=4,stride=2,padding=1,bias=False),
        nn.BatchNorm2d(256),
        nn.ReLU(True),
    
        nn.ConvTranspose2d(256,128,kernel_size=4,stride=2,padding=1,bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU(True),
    
        nn.ConvTranspose2d(128,64,kernel_size=4,stride=2,padding=1,bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(True),
    
        nn.ConvTranspose2d(64,3, kernel_size=4,stride=2,padding=1,bias=False),
        nn.Tanh()      
)

In [20]:
xb = torch.randn(batch_size, latent_size,1,1)
generated_images = generator(xb)
print(generated_images.shape)
show_images(generated_images)

In [21]:
generator = to_device(generator,device)
generator

In [22]:
def train_discriminator(real_images, opt_d):
    opt_d.zero_grad()
    
    #passing images through discriminator
    real_preds = discriminator(real_images)
    real_labels = torch.ones(real_images.size(0),1,device=device)
    real_loss = F.binary_cross_entropy(real_preds,real_labels)
    real_score = torch.mean(real_preds).item()
    
    #generateing fake images
    latent = torch.randn(batch_size,latent_size, 1,1,device=device)
    generated_images = generator(latent)
    
    #pass fake images through discriminator
    fake_labels = torch.zeros(generated_images.size(0),1,device=device)
    fake_preds = discriminator(generated_images)
    fake_loss = F.binary_cross_entropy(fake_preds,fake_labels)
    fake_score = torch.mean(fake_preds).item()
    
    #updating discriminator weights
    loss = real_loss + fake_loss
    loss.backward()
    opt_d.step()
    
    return loss, real_score, fake_score

In [23]:
def train_generator(opt_g):
   
    opt_g.zero_grad()
    
  
    latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
    generated_images = generator(latent)
  
    preds = discriminator(generated_images)
    targets = torch.ones(batch_size, 1, device=device)
    loss = F.binary_cross_entropy(preds, targets)
    
 
    loss.backward()
    opt_g.step()
    
    return loss.item()

In [24]:
from torchvision.utils import save_image

In [25]:
sample_dir = 'generated_cars'
os.makedirs(sample_dir, exist_ok=True)

In [26]:
def save_samples(index, latent_tensors, show=True):
    generated_images = generator(latent_tensors)
    fake_fname = 'generated-images-{0:0=4d}.png'.format(index)
    save_image(denorm(generated_images), os.path.join(sample_dir, fake_fname), nrow=8)
    print('Saving', fake_fname)
    if show:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(generated_images.cpu().detach(), nrow=8).permute(1, 2, 0))

In [27]:
fixed_latent = torch.randn(64, latent_size, 1, 1, device=device)

In [28]:
save_samples(0, fixed_latent)

In [29]:
from tqdm.notebook import tqdm
import torch.nn.functional as F

In [30]:
def fit(epochs, lr, start_idx=1):
    torch.cuda.empty_cache()
    
    # Losses & scores
    losses_g = []
    losses_d = []
    real_scores = []
    fake_scores = []
    
    # Create optimizers
    opt_d = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    opt_g = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    for epoch in range(epochs):
        for real_images, _ in tqdm(train_dl):
            # Train discriminator
            loss_d, real_score, fake_score = train_discriminator(real_images, opt_d)
            # Train generator
            loss_g = train_generator(opt_g)
            
        # Record losses & scores
        losses_g.append(loss_g)
        losses_d.append(loss_d)
        real_scores.append(real_score)
        fake_scores.append(fake_score)
        
        # Log losses & scores (last batch)
        print("Epoch [{}/{}], loss_g: {:.4f}, loss_d: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}".format(
            epoch+1, epochs, loss_g, loss_d, real_score, fake_score))
    
        # Save generated images
        save_samples(epoch+start_idx, fixed_latent, show=False)
    
    return losses_g, losses_d, real_scores, fake_scores

In [31]:
lr = 0.0002
epochs = 40

In [32]:
history = fit(epochs, lr)

In [37]:
# Save the model checkpoints 
genModel = torch.save(generator.state_dict(), 'G_cars.ckpt')
discModel = torch.save(discriminator.state_dict(), 'D_cars.ckpt')

In [38]:
from IPython.display import Image

In [39]:
Image('./generated_cars/generated-images-0001.png')

In [47]:
Image('./generated_cars/generated-images-0025.png')

In [42]:
Image('./generated_cars/generated-images-0040.png')

In [48]:
import cv2
import os

vid_fname = 'gans_cars_training.avi'

files = [os.path.join(sample_dir, f) for f in os.listdir(sample_dir) if 'generated' in f]
files.sort()



out = cv2.VideoWriter(vid_fname,cv2.VideoWriter_fourcc(*'XVID'), 1, (530,530))
[out.write(cv2.imread(fname)) for fname in files]
out.release()

In [51]:
modelGen = torch.load('G_cars.ckpt')


In [52]:
modelDisc = torch.load('D_cars.ckpt')

In [53]:
save_samples(41, fixed_latent)

In [54]:
save_samples(42, fixed_latent)

In [65]:
save_samples(46, torch.randn(32, 128, 1, 1, device=device))